In [1]:
import numpy as np
import pandas as pd

In [2]:
%run 0.1_Variables-and-Constants.ipynb

In [3]:
def result_df(__type="成績",__start="2015", __end="2025", __location=location_list):
    csv_dir = Path(base_dir, __type)
    csv_files = csv_dir.glob("**/*.CSV")

    # 空のDataFrameを作成
    df = pd.DataFrame()
    
    # カラム名
    if __type == "成績":
        columns = result_columns
    elif __type == "基準タイム":
        columns = time_columns
    elif __type == "配当":
        columns = payout_columns
    else:
        raise Exception
        

    # CSVファイルごとに処理
    for file in csv_files:
        year = "20" + file.stem[0:2]
        held = file.stem[2]
        location = file.stem[3]
        if (
            int(year) >= int(__start)
            and int(year) <= int(__end)
            and location in __location
        ):
            # 最適化: chunksizeを指定してメモリ効率良く処理
            chunk_iter = pd.read_csv(
                file,
                chunksize=10000,
                encoding="cp932",
                header=None,
                names=columns,
            )
            for chunk in chunk_iter:
                df = pd.concat([df, chunk], ignore_index=True)

    # 個別の前処理
    if __type == "成績":
        # 年、月、日からdateカラム（yymmdd）を生成
        df["レース日"] = (
            df["年"].astype(str).str.zfill(2)
            + df["月"].astype(str).str.zfill(2)
            + df["日"].astype(str).str.zfill(2)
        )
        
        # dateカラムをdatetime型に変換
        df["レース日"] = pd.to_datetime(df["レース日"], format="%y%m%d", yearfirst=True)
        
        # 不要なカラムを削除
        df.drop(
            columns=[
                "馬印",
                "レース印",
                "入線着順",
                "血統登録番号",
                "騎手コード",
                "調教師コード",
            ],
            inplace=True,
        )
    elif __type == "基準タイム":
        pattern = r"(?P<min>\d)\.(?P<sec>\d{2})\.(?P<tenth>\d)"
        out = df["基準タイム"].str.extract(pattern).astype(int)
        df["基準タイム(秒)"] = out["min"] * 60 + out["sec"] + out["tenth"] / 10
    elif __type == "配当":
        pass
    else:
        raise Exception
    return df

In [4]:
# Check DataFrame

In [5]:
df = result_df(__type='成績',__start="2021", __end="2024", __location=['東'])

In [6]:
df.sample(5)

,年,月,日,回次,場所,日次,レース番号,レース名,クラスコード,芝・ダ,...,馬主名,生産者名,父馬名,母馬名,母の父馬名,毛色,生年月日,単勝オッズ,PCI,レース日
19073,22,6,25,3,東京,7,11,江の島Ｓ･3勝,67,芝,...,サンデーレーシング,社台コーポレーショ,ロードカナロア,イリュミナンス,マンハッタンカフェ,鹿毛,170214,13.6,55.6,2022-06-25
4064,21,2,20,1,東京,7,12,２勝ｸﾗｽ,43,芝,...,キャロットファーム,社台コーポレーショ,スクリーンヒーロー,ユールフェスト,More Than Ready,青鹿,160130,5.1,51.1,2021-02-20
15083,24,11,10,5,東京,4,7,２勝ｸﾗｽ・牝,43,ダ,...,江馬由将,ノーザンファーム,ドゥラメンテ,ソーメニーウェイズ,Sightseeing,黒鹿,200122,2.2,53.3,2024-11-10
24609,24,6,9,3,東京,4,6,未勝利,7,芝,...,ノルマンディーサラ,土田ファーム,リオンディーズ,メルジェリーナ,ハービンジャー,青鹿,210213,277.7,48.3,2024-06-09
10008,23,6,10,3,東京,3,4,未勝利・牝,7,芝,...,河崎五市,中村雅明,ハクサンムーン,アクロスザシャイン,タニノギムレット,栗毛,200329,185.4,54.9,2023-06-10


In [7]:
df = result_df(__type='基準タイム',__start="2021", __end="2024", __location=['東'])

In [8]:
df.sample(5)

,レースID,基準タイム,開催名,基準タイム(秒)
804,05232C12,2.30.5,2023年2回東京12日目12R,150.5
1221,5235708,1.24.1,2023年5回東京7日目 8R,84.1
1083,5223311,2.26.0,2022年3回東京3日目11R,146.0
427,5213410,1.37.3,2021年3回東京4日目10R,97.3
703,05232406,1.38.1,2023年2回東京4日目 6R,98.1


In [9]:
df = result_df(__type='配当',__start="2021", __end="2024", __location=['東'])

In [10]:
df[['レースID','単勝配当1','複勝配当1', '複勝配当2','複勝配当3','ワイド配当1','ワイド配当2','ワイド配当3', '馬連配当1','馬単配当1','３連複配当1','３連単配当1']].sample(5)

,レースID,単勝配当1,複勝配当1,複勝配当2,複勝配当3,ワイド配当1,ワイド配当2,ワイド配当3,馬連配当1,馬単配当1,３連複配当1,３連単配当1
1539,05222704,1000,190,130,110,570,300,170,2810,5590,710,10570
383,5244812,780,320,610,360,2680,1480,2650,9300,12880,22790,108500
318,5244307,350,170,710,480,2130,810,3700,6540,10180,19690,93680
1000,5223705,350,160,260,190,580,520,780,1540,2510,2540,11270
204,5231201,420,180,1260,430,3530,1380,14170,13330,19310,86180,479600


In [4]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import matplotlib.pyplot as plt


plt.rcParams.update({
    "font.family": "Hiragino Sans",
    "axes.unicode_minus": False  # − が □ になるのを防ぐ
})